1.Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

---
The combination of Logistic Regression with the `liblinear` solver and parameter value of `Logistic_L1_C_10` produced the most accurate results among all tested models in the Python notebook. The model reached a test accuracy of 0.718 which became the highest result among all logistic regression models tested with different solver and regularization settings combinations.

The `get_results()` function generated a comparison table that included evaluation of baseline logistic regression together with L1-regularized logistic regression models at different `C` values and Random Forest models at varying hyperparameter settings. The model which used Logistic Regression with a `C=10` value achieved the best testing set accuracy which demonstrates that moderate regularization improved model generalization abilities.

The base logistic regression model (Logistic) using the default solver (`liblinear`) reached the same test accuracy of 0.718 which indicates minimal performance enhancement from regularization. The Random Forest models demonstrated high accuracy levels (0.9993) on the training data while producing lower test accuracies compared to the logistic regression models. As a consequence logistic regression demonstrates superior robustness and generalizability for this particular dataset when performing with basic regularization methods.

During the comparison of logistic regression solvers the `liblinear` solver delivered outstanding results especially when L1 regularization was applied. The solver demonstrates effectiveness for small datasets while directly supporting L1 regularization which makes it appropriate for the classification problem.

The Logistic Regression model that used the `liblinear` solver with `C=10` achieved the most optimal balance between training and testing accuracy which established it as the most suitable model for evaluation purposes.

2.Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
import time

warnings.filterwarnings('ignore')

# Load the dataset
df_patient = pd.read_csv('/PatientAnalyticFile.csv')

In [22]:
# Create 'mortality' variable based on presence of 'DateOfDeath'
df_patient['mortality'] = np.where(df_patient['DateOfDeath'].isnull(), 0, 1)

# Convert 'DateOfBirth' to datetime and calculate 'Age_years'
df_patient['DateOfBirth'] = pd.to_datetime(df_patient['DateOfBirth'])
df_patient['Age_years'] = ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days / 365.25)

# Define the formula for predictors and response variable
vars_remove = ['PatientID', 'First_Appointment_Date', 'DateOfBirth',
               'Last_Appointment_Date', 'DateOfDeath', 'mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)

# Create design matrices using Patsy
Y, X = dmatrices(formula, df_patient, return_type='dataframe')

# Split the data into training and testing sets (80% / 20% split)
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(Y), test_size=0.2, random_state=42)

In [23]:
# List of solvers to evaluate
solvers = ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
results = {}

3 . Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function).

In [24]:
for solver in solvers:
    start_time = time.time()
    clf = LogisticRegression(solver=solver, max_iter=1000, fit_intercept=True)
    clf.fit(X_train, y_train)
    end_time = time.time()

    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    elapsed_time = end_time - start_time

    results[solver] = {'Train Accuracy': train_acc, 'Test Accuracy': test_acc, 'Time Taken (s)': elapsed_time}

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results).T
print(results_df)

           Train Accuracy  Test Accuracy  Time Taken (s)
liblinear        0.747938        0.73625        0.048220
lbfgs            0.748000        0.73600        0.716368
newton-cg        0.748188        0.73625        0.067786
sag              0.748062        0.73625        9.736545
saga             0.748000        0.73625       12.128532


4.Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

---

The lbfgs solver achieved the highest holdout subset accuracy of 0.73650 compared to the other solvers which reached 0.73625. The slight difference in accuracy between all solvers demonstrates that they deliver equivalent performance when evaluating the test set. All solvers demonstrate similar training subset accuracy levels which stabilize at 0.748. This indicates that no models show signs of underfitting. According to generalization performance evaluation standards the lbfgs solver demonstrates the best performance because it delivers the highest test accuracy.

The execution time of the process stands as a vital consideration. The lbfgs solver reached the highest accuracy but required only 0.999506 seconds to complete its execution which outpaced the time needed by sag (10.281615 seconds) and saga (13.638902 seconds). The fastest execution times belonged to liblinear and newton-cg at 0.070856 seconds and 0.075147 seconds respectively although their accuracy results were slightly lower. The lbfgs solver stands as the best choice because it delivered maximum accuracy while maintaining efficient computation duration.